# Vibration Data Processing

### Industrial pump

The object of this notebook is the processing of vibration data acquired from a sensor monitoring an industrial pump in order to do identify the machine state from a measure.  

First, the data is processed to produce features that can be used in a ML model.
  
Then a Principla Component Analysis is run on the dataset to reduce the feature space.  
  
Finally a SVM model is trained to predict the state of the machine based on the vibration recording and tested on some samples.  
  

In [1]:
%matplotlib inline
from __future__ import print_function # for Python 2.7 compatibility
from IPython.display import display, HTML

import operator
import math
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import scipy.signal as sg

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from math import floor, log

In [2]:
# Load the csv data

df = pd.read_csv('./Input_data/DataSet/project_pump.csv', header=None, sep="[;]")
#df = pd.read_csv('./Input_data/DataSet/project_fan.csv', header=None, sep="[;]")

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


IOError: [Errno 2] No such file or directory: './Input_data/DataSet/project_pump.csv'

In [ ]:
# Convert the measures loaded as string in the last column as list of float
df[4] = df[4].apply(lambda x : map(float,x.strip("[]").split(", ")))

# Convert the timestamps into ms elapsed since the origin
df[0] = df[0].apply(lambda x : x-df[0][0])

#Verify that there is no null data in the set
df.isnull().any().sum()

In [ ]:
# Display part of the data loaded

def display_pd(some_df):
    display(HTML(some_df.to_html()))
    
display_pd(df.head(5)) # special display to see all columns
print('{} samples'.format(df.shape[0])) # Dimensions of the dataframe

In [ ]:
# Compute the constant baseline on the 50 first samples
mean_list = []
for i in range(50):
    mean_list.append(np.mean(df[4].iloc[i]))
baseline = np.mean(mean_list)
print('baseline measure : {} mV'.format(baseline))

In [ ]:
# Signal processing functions

def rem_baseline(signal):
    ''' Remove the baseline signal from sample measure
    '''
    signal -= baseline
    return signal
    
def calcFFT(epoch):
    ''' Calculates the FFT of the epoch signal. Removes the DC component.
    '''
    M = len(epoch)
    w = np.kaiser(M, 0.11)
    spectrum = np.fft.fft(w*epoch, axis=0)
    spectrum = np.abs(spectrum)
    spectrum[0]=0 # set the DC component to zero
    #S /= S.sum()

    return spectrum

def calcWelch(epoch):
    """Calculate the PSD using the Welch method.
    """
    len(epoch)
    samp_rate = 48188
    f, epoch = sg.welch(epoch, samp_rate)
    return f, epoch

def calcPSDPeak(epoch):
    """Calculate the PSD using the Welch method.
    """
    len(epoch)
    f, epoch = sg.welch(epoch, fs)
    return f, epoch

In [ ]:
class Signal:
    def __init__(self, signal):
        self.signal = signal
        self.signal = self.signal.apply(rem_baseline)
    def process(self, processor):
        processor.start_object(self.signal)
        return processor.process()

In [ ]:
class ProcessorSelector():
    def get_processor(self, transf):
        if transf == 'FFT':
            return FFT_Processor()
        elif transf == 'PSD':
            return PSD_Processor()

select = ProcessorSelector()

In [ ]:
class SignalProcessor:
    def process(self, signal, transf):
        processor = select.get_processor(transf)
        return signal.process(processor)
    
class FFT_Processor:
    def __init_(self):
        self._current_obj = None
    def start_object(self, signal):
        self._current_obj = signal
    def process(self):
        return self._current_obj.apply(calcFFT)

class PSD_Processor:
    def __init_(self):
        self._current_obj = None
    def start_object(self, signal):
        self._current_obj = signal
    def process(self):
        f_list, psd_list = [], []
        for i in range(len(self._current_obj)):
            f, psd = calcWelch(self._current_obj[i])
            f_list.append(f)
            psd_list.append(psd)
        f = pd.Series(f_list)
        psd = pd.Series(psd_list)
        return f, psd #self._current_obj.apply(calcWelch)

In [ ]:
signal = Signal(df[4].copy(deep=True))
processor = SignalProcessor()

fft = processor.process(signal, 'FFT')
f, psd = processor.process(signal, 'PSD')


Based on the analysis done, the selected features are the following:  
- Peaks values from the spectrum,
- Peaks values from the PSD,
- Mean,
- Variance.

For each sample, a total of 12 peaks are collected from the spectrum.
The peaks values for the spectrum are determined automatically for each sample in 5 ranges of frequency:
[8000, 12500], [3000, 6000], [0, 1000], [17000, 18500], [19000, 22000]
If there is no peak above the value predefined in each range compare to the rest of the signal, the value is set to 0 for this sample. 

Likewise, a total of 6 peaks are collected from the PSD, in 6 ranges of frequency:
[0, 2000], [2000, 6000], [6000, 10000], [10000, 15000], [15000, 19000], [19000, 23000]

In [ ]:
# Calculate the features

from sklearn.preprocessing import MinMaxScaler

features = ['fft_pk1', 'fft_pk2', 'fft_pk3', 'fft_pk4', 'fft_pk5', 'pk_PSD', 'mean', 'var']


peaks = df[4].copy(deep=True)

features_df = pd.DataFrame(np.zeros((df.shape[0], 19)))

for feat in features:
    if feat[:2] == 'ff':
        pk_nb = [0, 4, 3, 2, 1, 2][int(feat[-1])]-sum([0, 4, 3, 2, 1, 2][:int(feat[-1])-1])
        freq_range = [[0,400], [3000,6000],[9000,12500], [17000,18500], [19000,22000]][int(feat[-1])-1]
        
        if feat[-1] == '0':
            percent, dist, prom, mini = [89, 15, 0.9, 600]
        elif feat[-1] == '4':
            percent, dist, prom, mini = [89, 15, 6000, 600]
        else:
            percent, dist, prom, mini = [95, 200, 0.9, 600]
        
        for i in range(df[4].shape[0]):
            inf = int(freq_range[0]/float((df[3].iloc[i]/df[1].iloc[i])))
            sup = int(freq_range[1]/float((df[3].iloc[i]/df[1].iloc[i])))
            
            #if feat[-1] == '0':
            #    peaks[i] = np.array([])
            try:
                indexes, _ = sg.find_peaks(fft[i][inf:sup], height = max(mini,np.percentile(fft[i][inf:sup], percent)), distance=dist, prominence=prom)
                #peaks[i] = np.concatenate([peaks[i],np.zeros(pk_nb)])
                for j in range(min(pk_nb, len(indexes))):
                    features_df.iloc[i,j+[0, 4, 3, 2, 1, 2][int(feat[-1])-1]] = fft[i][indexes[j]]
                    #peaks[i][-pk_nb+j] = fft[i][indexes[j]]
            except:
                continue
                #peaks[i] = np.concatenate([peaks[i],np.zeros(pk_nb)])
        
    elif feat == 'pk_PSD':
        list_indexes = []
        percent, dist, prom, mini = [85, 0, 0, 0.0001]
        for i in range(df[4].shape[0]):
            #peaks[i] = np.array([])
            indexes, _ = sg.find_peaks(psd[i], height = max(mini, np.percentile(psd[i], percent)))
            #peak_freq = [0,0,0,0,0,0]
            for j in range(1, 7):
                freq_range = [0, 2000, 6000, 10000, 15000, 19000, 23000]
                try:
                    features_df.iloc[i, j+12] = max(psd[i][indexes[(f.iloc[i][indexes]>freq_range[j-1]) & (f.iloc[i][indexes]<freq_range[j])]])
                    #peak_freq[j] = max(psd[i][indexes[(f.iloc[i][indexes]>freq_range[j-1]) & (f.iloc[i][indexes]<freq_range[j])]])
                except:
                    continue
            #peaks[i] = np.concatenate([peaks[i],peak_freq])
            list_indexes.append(len(indexes))
    elif feat == 'mean':
        for i in range(df[4].shape[0]):
            features_df.iloc[i, 17] = np.mean(df[4].iloc[i])
            #peaks[i] = np.concatenate([peaks[i],[np.mean(df[4].iloc[i])]])
    elif feat == 'var':
        for i in range(df[4].shape[0]):
            features_df.iloc[i, 18] = np.var(df[4].iloc[i])
            #peaks[i] = np.concatenate([peaks[i],[np.var(df[4].iloc[i])]])


features_df.fillna(0)
# Rescale the features to (0,1)
for i in range(19):
    min_max_scaler = MinMaxScaler()
    x = features_df.iloc[:,i].values.reshape(1,-1)
    x = np.array(features_df.iloc[:,i].values).reshape(x.shape[1],1)
    x_scaled = min_max_scaler.fit_transform(x)
    features_df.iloc[:,i] = pd.DataFrame(x_scaled)
    

#peaks.values.shape
#print(peaks.values[350])
#min_max_scaler = MinMaxScaler()
#peaks_scaled = min_max_scaler.fit_transform(peaks.values.reshape(-1, 1))
#df = pandas.DataFrame(peaks_scaled)

In [ ]:
from sklearn.decomposition import PCA

nb_comp = 7
pca = PCA(n_components=nb_comp)
principalComponents = pca.fit_transform(features_df)

print('PCA Components Explained Variance')
print('Components Variance: {}'.format(pca.explained_variance_ratio_))
print('Total Explained Variance for {} components: {}%'.format(nb_comp, np.round(100.*sum(pca.explained_variance_ratio_),2)))

principal_df = pd.DataFrame(data = principalComponents
             , columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7'])

From the PCA analysis, 97.7% of the variance of the data can be represented by 7 components.  
We will use this PC reduction for the model.

It is possible to test this set of features for finding out to which phase a sample belongs.
To do so, we define the phases and select samples that we annotate for training a model.

We will use a SVM model for this test.

In [ ]:
# Annotate the samples
steps = [181,538,709,735]
categories = np.zeros(df.shape[0])
for step in steps:
    for j in range(step,df.shape[0]):
        categories[j] += 1



principal_df['cat'] = categories

In [ ]:
#Create the test set
test_samples = [50, 75, 100, 200, 300, 400, 500, 600, 650, 709]
test_set_df = principal_df.iloc[test_samples].copy(deep=True)
test_set_df = test_set_df.drop(['cat'], axis = 1)
#Create the train set
train_set_df = principal_df.drop([50, 75, 100, 200, 300, 400, 500, 600, 650, 709], axis=0).copy(deep=True)
print(test_set_df.shape, train_set_df.shape)

In [ ]:
from sklearn import svm

X = train_set_df[['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7']]
Y = train_set_df['cat']
clf = svm.SVC()
clf.fit(X, Y)

pred = clf.predict(test_set_df)
print(pred)

This approach provide some interesting resutls already.  
With a sensor placed on the same machine, we can verify what is the regime of the machine and probably detect some undesired situations.  
This requires to have data annotated beforehand.  